In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output

In [26]:
data = pd.read_csv('incomes_data.csv')

In [27]:
data.head()

,odor,sar,hariltsagch_dans,hariltsagch_ner,utga,uldegdel,orlogo
0,9,5,0,0,КАРТ ЗАХИАЛГЫН ҮНЭ/ ХАТАНБОЛД-С,5000.000,5000.000
1,9,5,457054690,амгалан ганбат,EB -4 sar tsalin,134000.000,134000.000
2,16,5,457054690,амгалан ганбат,EB -hdmi converter,10150.000,7000.000
3,17,5,0,949610xxxxxx4357,00000064:ATM P2P Credit,30100.000,20000.000
4,27,5,0,бхтт-ны бага дүнтэй гүйлгээний авлага,computer : 050000-ХААН Банк(5374373991-БОЛД У...,130000.000,130000.000


In [31]:
train = data.iloc[:,0:6]
y_train = data.orlogo.values
train.head()

,odor,sar,hariltsagch_dans,hariltsagch_ner,utga,uldegdel
0,9,5,0,0,КАРТ ЗАХИАЛГЫН ҮНЭ/ ХАТАНБОЛД-С,5000.000
1,9,5,457054690,амгалан ганбат,EB -4 sar tsalin,134000.000
2,16,5,457054690,амгалан ганбат,EB -hdmi converter,10150.000
3,17,5,0,949610xxxxxx4357,00000064:ATM P2P Credit,30100.000
4,27,5,0,бхтт-ны бага дүнтэй гүйлгээний авлага,computer : 050000-ХААН Банк(5374373991-БОЛД У...,130000.000


In [32]:
print(train.shape)
train = pd.get_dummies(train)
print(train.shape)

(128, 6)
(128, 96)


In [58]:
train.columns

Index(['odor', 'sar', 'hariltsagch_dans', 'uldegdel', 'hariltsagch_ner_0',
       'hariltsagch_ner_949610xxxxxx4357', 'hariltsagch_ner_амарсайхан одбаяр',
       'hariltsagch_ner_амгалан ганбат',
       'hariltsagch_ner_бага дүнтэй гүйлгээний хоорондын тооцоо',
       'hariltsagch_ner_бат-оргил батбуян',
       'hariltsagch_ner_бат-эрдэнэ сүх-очир',
       'hariltsagch_ner_бхтт-ны бага дүнтэй гүйлгээний авлага',
       'hariltsagch_ner_дэнцү дата артист монгол гхоххк',
       'hariltsagch_ner_лэндмн ббсб', 'hariltsagch_ner_маргад алтангэрэл',
       'hariltsagch_ner_мост мони гүйлгээний түр данс / mnt',
       'hariltsagch_ner_хатанболд батзориг', 'hariltsagch_ner_цэлмэн төгсбаяр',
       'hariltsagch_ner_эрдэнэ-очир мөнхзул',
       'utga_(3749928) НӨАТ-ын 2018 оны урамшуулал  : 910000-НЄАТ буцаан олголт(3402031-НӨАТ)-> 040000-(423009374-ЭРДЭНЭОЧИР МӨНХЗУЛ)',
       'utga_00000064:ATM P2P Credit', 'utga_AUG SALARY FROM DDAM LLC',
       'utga_Bodygram delbee  : 050000-ХААН Банк(504146

In [33]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [34]:
n_folds = 6

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [35]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [36]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [37]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [38]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [39]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [40]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [41]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 190166.1339 (77026.7302)



In [42]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 185413.9963 (84477.3882)



In [43]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: 189110.5166 (142365.3779)



In [44]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 105884.9516 (56424.4115)



In [45]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 95793.6370 (54311.4381)



In [46]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: 118907.6497 (49895.0578)



In [47]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [48]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 146667.0163 (85603.7815)



In [49]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [50]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 103532.7558 (51799.4910)


In [51]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [53]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
# stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

45135.253195633784


In [56]:
train.columns

Index(['odor', 'sar', 'hariltsagch_dans', 'uldegdel', 'hariltsagch_ner_0',
       'hariltsagch_ner_949610xxxxxx4357', 'hariltsagch_ner_амарсайхан одбаяр',
       'hariltsagch_ner_амгалан ганбат',
       'hariltsagch_ner_бага дүнтэй гүйлгээний хоорондын тооцоо',
       'hariltsagch_ner_бат-оргил батбуян',
       'hariltsagch_ner_бат-эрдэнэ сүх-очир',
       'hariltsagch_ner_бхтт-ны бага дүнтэй гүйлгээний авлага',
       'hariltsagch_ner_дэнцү дата артист монгол гхоххк',
       'hariltsagch_ner_лэндмн ббсб', 'hariltsagch_ner_маргад алтангэрэл',
       'hariltsagch_ner_мост мони гүйлгээний түр данс / mnt',
       'hariltsagch_ner_хатанболд батзориг', 'hariltsagch_ner_цэлмэн төгсбаяр',
       'hariltsagch_ner_эрдэнэ-очир мөнхзул',
       'utga_(3749928) НӨАТ-ын 2018 оны урамшуулал  : 910000-НЄАТ буцаан олголт(3402031-НӨАТ)-> 040000-(423009374-ЭРДЭНЭОЧИР МӨНХЗУЛ)',
       'utga_00000064:ATM P2P Credit', 'utga_AUG SALARY FROM DDAM LLC',
       'utga_Bodygram delbee  : 050000-ХААН Банк(504146

In [55]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
# xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

ValueError: feature_names may not contain [, ] or <

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
# lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))